# Build a vocabulary

In [1]:
import sys
sys.path.append('..')

In [2]:
import sqlite3
from collections import Counter, defaultdict, namedtuple
from pathlib import Path
from zipfile import ZipFile

import chardet
import nltk
import pandas as pd
import regex as re
from tqdm import tqdm

from digi_leap.pylib import db
from digi_leap.pylib import vocab

In [3]:
DATA = Path('..') / 'data' / 'raw'

IDIGBIO = DATA / 'occurrence_raw_idigbio_2021-02.sqlite3.db'
ITIS = DATA / 'ITIS.sqlite'
NLTK = DATA / 'nltk_corpora'
VOCAB = vocab.VOCAB_DB

In [4]:
WORDS = Counter()

## Common functions

In [5]:
def text_to_vocab(text):
    words = re.findall(r'\p{L}+', text)
    WORDS.update(words)

## Get NLTK corpora

In [6]:
SKIPS = {'README', 'LICENSE'}

In [7]:
def read_nltk(zip_file):
    print(f'\n{zip_file}')
    zip_path = NLTK / zip_file
    stem = zip_path.stem
    with ZipFile(zip_path) as zippy:
        for info in zippy.infolist():
            if Path(info.filename).stem in SKIPS or info.is_dir():
                continue
            print(info.filename)
            for ln in zippy.open(info.filename):
                enc = chardet.detect(ln)['encoding']  # Slow
                ln = ln.decode(enc)
                text_to_vocab(ln)

In [8]:
read_nltk('words.zip')
read_nltk('gazetteers.zip')
read_nltk('gutenberg.zip')
read_nltk('names.zip')
read_nltk('webtext.zip')


words.zip
words/en
words/en-basic

gazetteers.zip
gazetteers/caprovinces.txt
gazetteers/countries.txt
gazetteers/isocountries.txt
gazetteers/mexstates.txt
gazetteers/nationalities.txt
gazetteers/uscities.txt
gazetteers/usstateabbrev.txt
gazetteers/usstates.txt

gutenberg.zip
gutenberg/austen-emma.txt
gutenberg/austen-persuasion.txt
gutenberg/austen-sense.txt
gutenberg/bible-kjv.txt
gutenberg/blake-poems.txt
gutenberg/bryant-stories.txt
gutenberg/burgess-busterbrown.txt
gutenberg/carroll-alice.txt
gutenberg/chesterton-ball.txt
gutenberg/chesterton-brown.txt
gutenberg/chesterton-thursday.txt
gutenberg/edgeworth-parents.txt
gutenberg/melville-moby_dick.txt
gutenberg/milton-paradise.txt
gutenberg/shakespeare-caesar.txt
gutenberg/shakespeare-hamlet.txt
gutenberg/shakespeare-macbeth.txt
gutenberg/whitman-leaves.txt

names.zip
names/female.txt
names/male.txt

webtext.zip
webtext/firefox.txt
webtext/grail.txt
webtext/overheard.txt
webtext/pirates.txt
webtext/singles.txt
webtext/wine.txt


## Get ITIS data

In [9]:
def itis_names():
    sql = """
        select expert as name from experts
        union select reference_author from publications
        union select shortauthor from strippedauthor
        union select short_author from taxon_authors_lkp
        """
    for row in tqdm(db.rows_as_dicts(ITIS, sql)):
        text_to_vocab(row['name'])

In [10]:
def itis_taxa():
    # Data is normalized so don't bother with union "all"
    sql = """
        select unit_name1 as name from taxonomic_units where unit_name1 is not null
        union select unit_name2 from taxonomic_units where unit_name2 is not null
        union select unit_name3 from taxonomic_units where unit_name3 is not null
        union select unit_name4 from taxonomic_units where unit_name4 is not null
        """
    for row in tqdm(db.rows_as_dicts(ITIS, sql)):
        text_to_vocab(row['name'])

In [11]:
def itis_vernaculars():
    sql = """select vernacular_name from vernaculars"""
    for row in tqdm(db.rows_as_dicts(ITIS, sql)):
        text_to_vocab(row['vernacular_name'])

In [12]:
itis_names()
itis_taxa()
itis_vernaculars()

100%|███████████████████████████████████████████████████████████████████████████████| 131983/131983 [00:00<00:00, 273793.25it/s]


## Get iDigBio data

In [13]:
TAXON_COLS = """
    accepted_name_usage class family genus group
    higher_classification infraspecific_epithet
    kingdom order original_name_usage phylum
    previous_identifications scientific_name
    specific_epithet subgenus taxon_rank
    verbatim_taxon_rank vernacular_name parent_name_usage
    verbatim_scientific_name
    """.split()

NAME_COLS = """
    georeferenced_by identified_by location_according_to
    name_according_to recorded_by scientific_name_authorship
    record_entered_by 
    """.split()

PLACE_COLS = """
    continent country country_code county island
    island_group municipality higher_geography
    state_province verbatim_coordinate_system
    verbatim_coordinates verbatim_elevation
    verbatim_latitude verbatim_longitude water_body
    georeference_protocol verbatim_srs
    """.split()

WORD_COLS = """
    dataset_name dwc_rights_holder dwc_verbatim_event_date
    establishment_means event_remarks georeference_remarks
    georeference_sources habitat identification_remarks
    information_withheld life_stage locality location_remarks
    occurrence_remarks organism_remarks
    georeference_verification_status name_published_in
    owner_institution_code preparations reproductive_condition
    rights_holder sampling_protocol sex taxon_remarks
    taxonomic_status type_status verbatim_event_date
    verbatim_locality
    """.split()

COLUMNS = TAXON_COLS + NAME_COLS + PLACE_COLS + WORD_COLS

In [14]:
def idigbio_data():
    columns = [f"`{c}`" for c in COLUMNS]
    sql = f"""select {', '.join(columns)} from occurrence_raw"""
    with sqlite3.connect(IDIGBIO) as cxn:
        cxn.row_factory = sqlite3.Row
        for row in tqdm(cxn.execute(sql)):
            row = dict(row)
            for col in COLUMNS:
                text_to_vocab(row[col])

In [15]:
idigbio_data()

127261791it [8:53:18, 3977.17it/s]


## Write to the vocab database

In [16]:
def to_vocab_db():
    db.create_vocab_table(VOCAB, drop=True)
    batch = list(WORDS.items())
    db.insert_vocabulary_words(VOCAB, batch)

In [17]:
to_vocab_db()

## Look at the character set

The spell checker does not depend on the character set but it is interesting to see what's in the DB.

In [18]:
with sqlite3.connect(VOCAB) as cxn:
    sql = 'select * from vocab'
    WORDS = {r[0]: r[1] for r in cxn.execute(sql)}

In [19]:
chars = set()
for word, freq in WORDS.items():
    chars |= set(word)

print(sorted(chars))

['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', 'ª', 'µ', 'º', 'À', 'Á', 'Â', 'Ã', 'Ä', 'Å', 'Æ', 'Ç', 'È', 'É', 'Ê', 'Ë', 'Ì', 'Í', 'Î', 'Ï', 'Ð', 'Ñ', 'Ò', 'Ó', 'Ô', 'Õ', 'Ö', 'Ø', 'Ù', 'Ú', 'Û', 'Ü', 'Ý', 'Þ', 'ß', 'à', 'á', 'â', 'ã', 'ä', 'å', 'æ', 'ç', 'è', 'é', 'ê', 'ë', 'ì', 'í', 'î', 'ï', 'ð', 'ñ', 'ò', 'ó', 'ô', 'õ', 'ö', 'ø', 'ù', 'ú', 'û', 'ü', 'ý', 'þ', 'ÿ', 'Ā', 'ā', 'ă', 'ą', 'Ć', 'ć', 'Ĉ', 'ĉ', 'Ċ', 'ċ', 'Č', 'č', 'Ď', 'ď', 'Đ', 'đ', 'Ē', 'ē', 'Ĕ', 'ĕ', 'Ė', 'ė', 'ę', 'Ě', 'ě', 'ĝ', 'ğ', 'Ġ', 'ġ', 'ģ', 'ĥ', 'Ħ', 'ħ', 'ĩ', 'Ī', 'ī', 'ĭ', 'į', 'İ', 'ı', 'Ķ', 'ķ', 'ĺ', 'Ļ', 'ļ', 'Ľ', 'ľ', 'Ŀ', 'Ł', 'ł', 'ń', 'ņ', 'Ň', 'ň', 'ŉ', 'ŋ', 'Ō', 'ō', 'ŏ', 'Ő', 'ő', 'Œ', 'œ', 'ŕ', 'Ř', 'ř', 'Ś', 'ś', 'Ŝ', 'ŝ', 'Ş', 'ş', 'Š', 'š', 'Ţ', 'ţ', 'ť', 'ũ', 'Ū', 'ū', 'Ŭ', 'ŭ', 'Ů',

## Create a table to hold the misspelling mappings

In [4]:
SPELLS = {}

In [5]:
sql = """ select word, freq from vocab where length(word) >= 3 and freq >= 5 """

with sqlite3.connect(VOCAB) as cxn:
    for term in tqdm(cxn.execute(sql)):
        word, freq = term
        freq = -freq

        SPELLS[word] = (0, freq, word)

        for i in range(len(word)):
            miss1 = word[:i] + word[i+1:]
            new1 = (1, freq, word)
            spell1 = SPELLS.get(miss1, (99, 99, ''))

            if spell1 > new1:
                SPELLS[miss1] = new1

            for j in range(len(miss1)):
                miss2 = miss1[:j] + miss1[j+1:]
                new2 = (2, freq, word)
                spell2 = SPELLS.get(miss2, (99, 99, ''))

                if spell2 > new2:
                    SPELLS[miss2] = new2

1945481it [01:56, 16711.96it/s]


In [6]:
len(SPELLS)

63830356

In [7]:
SPELLS = [(k, v[2], v[0], -v[1]) for k, v in SPELLS.items()]

In [8]:
db.create_misspellings_table(VOCAB, drop=True)
db.insert_misspellings(VOCAB, SPELLS)